<a href="https://colab.research.google.com/github/pranaya-mathur/Deep-Learning-Projects/blob/master/ElasticSearch_on_Colab_7th_Sep_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,elasticsearch,inference,preprocessing] from https://files.pythonhosted.org/packages/a3/e3/8a7ebb563df04300c9e2387ac31e8ffda809b21f276faa838b32cf4a74a7/farm_haystack-1.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.7.0 from https://files.pythonhosted.org/packages/08/2f/62e9e455c4cf183ebf6672c47befabb6a36a37a8cdc86256a5632d6ace10/canals-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c6

In [2]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [3]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [4]:
import time

time.sleep(30)

In [5]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [6]:
import requests
res = requests.get('http://localhost:9200/_cluster/health')

res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 2,
 'active_shards': 2,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.0}

In [7]:
import json

with open('dev.json', 'r') as f:
    squad = json.load(f)

In [ ]:
squad

In [20]:
squad_docs = []

for sample in squad:
    squad_docs.append({
        'content': sample['context']
    })

In [22]:
document_store.write_documents(squad_docs)

In [24]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store)

In [25]:
len(squad)

16209

In [26]:
query = "Physics is a very abstract subject"

retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'With modern insights into quantum mechanics and technology that can accelerate particles close to the speed of light, particle physics has devised a Standard Model to describe forces between particles smaller than atoms. The Standard Model predicts that exchanged particles called gauge bosons are the fundamental means by which forces are emitted and absorbed. Only four main interactions are known: in order of decreasing strength, they are: strong, electromagnetic, weak, and gravit

In [28]:
res = requests.post('http://localhost:9200/document/_delete_by_query',
                    json={
                        'query': {
                            'match_all': {}
                        }
                    })

res.json()

{'took': 301,
 'timed_out': False,
 'total': 1204,
 'deleted': 1204,
 'batches': 2,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [29]:
res = requests.get('http://localhost:9200/document/_count')

res.json()

{'count': 0,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [30]:
# create list of contexts (we cannot do this using current dictionary format)
contexts = [sample['context'] for sample in squad]

# convert to set to remove duplicates, then back to list
contexts = list(set(contexts))

# convert back to dictionary format we need
squad_docs = [{'content': sample} for sample in contexts]

In [31]:
document_store.write_documents(squad_docs)

In [32]:
retriever = TfidfRetriever(document_store)

In [ ]:
retriever.retrieve(query)

In [36]:
# import BM25 retriever
from haystack.nodes import BM25Retriever

# intialize
retriever = BM25Retriever(document_store)

# and query
retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': 0.7329360380503047, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'A computational problem can be viewed as an infinite collection of instances together with a solution for every instance. The input string for a computational problem is referred to as a problem instance, and should not be confused with the problem itself. In computational complexity theory, a problem refers to the abstract question to be solved. In contrast, an instance of this problem is a rather concrete utterance, which can serve as the input for a decision probl